# Mount GDrive & Import Packages

In [32]:
# Importing useful packages
import cv2
import os
import re
import glob
import numpy as np
import pandas as pd
from imutils import paths

import time
import random
import scipy

# Visualization
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import animation
from mpl_toolkits import mplot3d
from IPython.display import HTML
import seaborn as sns
sns.set()

# For Kalman Filter
from numpy import dot, zeros, eye
from copy import deepcopy
from math import log, exp, sqrt
import sys
import scipy.linalg as linalg
from filterpy.stats import logpdf
from filterpy.common import pretty_str, reshape_z
from filterpy.common import Q_discrete_white_noise

# Extended Kalman Filter

In [33]:
# Importing data
# yolo = 'YOLOv3-tiny'
yolo = 'YOLOv4'

if yolo == 'YOLOv3-tiny':
    data = pd.read_csv("/home/icodes/Abel/Optimisasi_Tugas_Akhir/Object_Matching/Tuning_EKF_GA/EKF-yolov3-tiny.txt", sep=" ", header=None)
elif yolo == 'YOLOv4':
    data = pd.read_csv("/home/icodes/Abel/Optimisasi_Tugas_Akhir/Object_Matching/Tuning_EKF_GA/EKF-yolov4.txt", sep=" ", header=None)
data = data.drop([6], axis=1)
data.columns = ["time", "id", "conf", "u", "v", "z"]
data.head()

,time,id,conf,u,v,z
0,0.00000,0,0.999023,384,302,6.500
1,1.72977,0,0.998535,394,284,8.729
2,1.83965,0,0.998535,396,282,8.487
3,1.93588,0,0.997559,398,282,8.487
4,2.06521,0,0.997559,398,282,8.487


## Sensor Noise Covariance - EKF

In [34]:
time_diff = np.empty((len(data), 1))
time_data = data[['time']].to_numpy()
for i in range(len(data)):
  try:
    time_diff[i, 0] = time_data[i+1, 0] - time_data[i, 0]
  except:
    print("end of dataset reached")

end of dataset reached


In [35]:
time_diff = pd.DataFrame(time_diff)
time_diff.head()

,0
0,1.72977
1,0.10988
2,0.09623
3,0.12933
4,0.07864


In [36]:
time_diff.mean()

0    0.102191
dtype: float64

In [37]:
sensor_data = data.copy()
sensor_data = sensor_data[["u", "v", "z"]]
sensor_data.head()

,u,v,z
0,384,302,6.500
1,394,284,8.729
2,396,282,8.487
3,398,282,8.487
4,398,282,8.487


In [38]:
R = sensor_data.cov()
R = R.to_numpy()
print("Sensor Noise Covariance Matrix R : \n", R)

Sensor Noise Covariance Matrix R : 
 [[5442.37141372 -136.82326997  -26.00453956]
 [-136.82326997  730.23479358 -113.43366341]
 [ -26.00453956 -113.43366341   20.00062328]]


## Class Inisialization

In [39]:
class ExtendedKalmanFilter():
  def __init__(self, yolo, Q = None, dim_x = 6, dim_z = 3, dim_u = 0):
        self.dim_x = dim_x
        self.dim_z = dim_z
        self.dim_u = dim_u

        # Initial state (x, y, z, vx, vy, vz, u, v, udot, vdot)
        self.x = zeros((dim_x, 1))  # state

        # Initial sensor noise covariance matrix

        if yolo == 'YOLOv3-tiny':
            self.R = np.array([[1.16004, -2.22669e-01, -2.43000e-03],
                               [-2.22669e-01, 2.45931, -0.00549840],
                               [-2.43000e-03, -0.00549840, 0.0921667]])
        elif yolo == 'YOLOv4':
            self.R = np.array([[0.871042, 0.0258208, 0.0182336],
                               [0.0258208, 0.196295, 0.01083142],
                               [0.0182336, 0.01083142, 0.112137]])

        self.P = eye(dim_x)  # uncertainty covariance
        self.B = 0  # control transition matrix
        self.F = np.eye(dim_x)  # state transition matrix
        
        # process uncertainty
        if Q is not None:
          self.Q = Q
        else:
          self.Q = eye(dim_x)
        
        # measurement
        z = np.array([None] * self.dim_z)
        self.z = reshape_z(z, self.dim_z, self.x.ndim)

        # gain and residual are computed during the innovation step. We
        # save them so that in case you want to inspect them for various
        # purposes
        self.K = np.zeros(self.x.shape)  # kalman gain
        self.y = zeros((dim_z, 1)) # residual
        self.S = np.zeros((dim_z, dim_z))  # system uncertainty
        self.SI = np.zeros((dim_z, dim_z))  # inverse system uncertainty

        # identity matrix. Do not alter this.
        self._I = np.eye(dim_x)

        self._log_likelihood = log(sys.float_info.min)
        self._likelihood = sys.float_info.min
        self._mahalanobis = None

        # these will always be a copy of x,P after predict() is called
        self.x_prior = self.x.copy()
        self.P_prior = self.P.copy()

        # these will always be a copy of x,P after update() is called
        self.x_post = self.x.copy()
        self.P_post = self.P.copy()

  def predict_x(self, u=0):
        """
        Predicts the next state of X. If you need to
        compute the next state yourself, override this function. You would
        need to do this, for example, if the usual Taylor expansion to
        generate F is not providing accurate results for you.
        """
        # print("F = ", self.F)
        # print("x_prev = ", self.x)
        # print("F dot x = ", dot(self.F, self.x))
        self.x = dot(self.F, self.x) + dot(self.B, u)

  def predict(self, dt, x_prev, P_prev, u=0):
        # Get previous state & covariance
        self.x = x_prev.reshape(self.dim_x, 1)
        self.P = P_prev

        # Update the state transition matrix
        self.F = np.array([[1, 0, 0, dt, 0, 0],
                           [0, 1, 0, 0, dt, 0],
                           [0, 0, 1, 0, 0, dt],
                           [0, 0, 0, 1, 0, 0],
                           [0, 0, 0, 0, 1, 0],
                           [0, 0, 0, 0, 0, 1]])

        self.predict_x(u)
        self.P = dot(self.F, self.P).dot(self.F.T) + dt * self.Q

        # save prior
        self.x_prior = np.copy(self.x)
        self.P_prior = np.copy(self.P)

        # print("x_pred = ", self.x_prior)
        # print("P_pred = ", self.P_prior)

        x_predicted = self.x_prior.copy()
        x_predicted = x_predicted.reshape(self.dim_x, 1).ravel()

        return x_predicted, self.P_prior

  def HJacobian(self, x, fx, fy):
        # print("fx = ", fx)
        # print("z = ", x[2][0])
        #The Jacobian Measurement Matrix (H_j)
        H_j = np.array([[fx/x[2][0], 0, (-fx*x[0][0])/(x[2][0]**2), 0, 0, 0],
                        [0, fy/x[2][0], (-fy*x[1][0])/(x[2][0]**2), 0, 0, 0],
                        [0, 0, 1, 0, 0, 0]])
        return H_j

  def Hx(self, x, cx, fx, cy, fy):
        # The Non-Linear Measurement Model
        H_x = np.array([[fx*(x[0][0]/x[2][0]) + cx],
                       [fy*(x[1][0]/x[2][0]) + cy], 
                       [x[2][0]]])
        return H_x

  def update(self, z, cx, fx, cy, fy, x_pred, P_pred, r, R=None, residual=np.subtract):

        # Get prediction result (states & state covariance)
        self.x = x_pred.reshape(self.dim_x, 1)
        self.P = P_pred

        # Update sensor variance
        # self.sigma_z = 0.001063 + 0.0007278 * z[2] + 0.003949 * (z[2] ** 2)
        self.sigma_z = 3e-05*(z[2]**2.9826)
        
        # Update sensor noise covariance matrix
        self.R[2, 2] = self.sigma_z

        # Measurement
        z = z.reshape(self.dim_z, 1)
        if z is None:
            self.z = np.array([[None] * self.dim_z]).T
            self.x_post = self.x.copy()
            self.P_post = self.P.copy()
            return

        if R is None:
            R = self.R
        elif np.isscalar(R):
            R = eye(self.dim_z) * R

        if np.isscalar(z) and self.dim_z == 1:
            z = np.asarray([z], float)

        H = self.HJacobian(self.x, fx, fy)

        PHT = dot(self.P, H.T)
        self.S = dot(H, PHT) + R
        self.K = PHT.dot(linalg.inv(self.S))

        hx = self.Hx(self.x, cx, fx, cy, fy)
        self.y = residual(z, hx)

        if r < 0.5:
          # Update Step
          # print("Update Step")
          self.x = self.x + dot(self.K, self.y)
          # P = (I-KH)P(I-KH)' + KRK' is more numerically stable
          # and works for non-optimal K vs the equation
          # P = (I-KH)P usually seen in the literature.
          I_KH = self._I - dot(self.K, H)
          self.P = dot(I_KH, self.P).dot(I_KH.T) + dot(self.K, R).dot(self.K.T)
        else:
          # print("No update")
          # print(self.x)
          pass

        # set to None to force recompute
        self._log_likelihood = None
        self._likelihood = None
        self._mahalanobis = None

        # save measurement and posterior state
        # self.z = deepcopy(z)
        self.x_post = self.x.copy()
        self.P_post = self.P.copy()

        x_updated = self.x_post.copy()
        x_updated = x_updated.reshape(self.dim_x, 1).ravel()

        residu = self.y.copy()
        residu = residu.reshape(self.dim_z, 1).ravel()

        return x_updated, self.P_post, residu

## Simulation

In [40]:
# Depth Intrinsics
cx = 320.483
cy = 240.989
fx = 382.727
fy = 382.727

In [41]:
# Data preparation
data = data.drop(['id'], axis = 1)
data = data.drop(['conf'], axis = 1)
print(data)
data = data.to_numpy()

         time    u    v      z
0     0.00000  384  302  6.500
1     1.72977  394  284  8.729
2     1.83965  396  282  8.487
3     1.93588  398  282  8.487
4     2.06521  398  282  8.487
..        ...  ...  ...    ...
255  26.06580  502  296  7.105
256  26.20600  506  290  7.023
257  26.30710  506  290  7.274
258  26.42670  506  293  7.834
259  26.56970  504  290  8.147

[260 rows x 4 columns]


In [42]:
def simulate(yolo, cx, fx, cy, fy, Q, data, P_0 = None):
  # Initialization
  x_0 = np.array([1, 1, 1, 0, 0, 0])
  
  if P_0 is None:
    P_0 = np.ones((6, 6))

  x_pred = np.empty((len(data), x_0.shape[0]))
  P_pred = []

  x_upd = np.empty((len(data), x_0.shape[0]))
  P_upd = []

  residual = np.empty((len(data), 3))
  total_velocity = 0

  EKF = ExtendedKalmanFilter(yolo, Q=Q, dim_x=6, dim_z=3)
  
  for i in range(len(data)):
    if i == 0:
      # Predict Step
      x_pred[i, :] = x_0
      P_pred.append(P_0)

      # Measurement
      z = data[i, 1:4]

      # Update Step
      r = np.random.rand()
      x_temp, P_temp, r_temp = EKF.update(z, cx, fx, cy, fy, x_pred[i, :], P_pred[-1], r)
      residual[i, :] = r_temp
      x_upd[i, :] = x_temp
      P_upd.append(P_temp)
    
    else:
      # Predict Step
      dt = data[i, 0] - data[i-1, 0]
      x_temp, P_temp = EKF.predict(dt, x_upd[i-1, :], P_upd[-1])
      x_pred[i, :] = x_temp
#       print(x_temp[3])
      P_pred.append(P_temp)
      total_velocity = total_velocity + np.sum(np.absolute(x_temp[3:6]))

      # Measurement
      z = data[i, 1:4]

      # Update Step
      r = np.random.rand()
      x_temp, P_temp, r_temp = EKF.update(z, cx, fx, cy, fy, x_pred[i, :], P_pred[-1], r)
      residual[i, :] = r_temp
      x_upd[i, :] = x_temp
      P_upd.append(P_temp)

  residual_u = np.sum(np.absolute(residual[i, 0]), axis = 0)
  residual_v = np.sum(np.absolute(residual[i, 1]), axis = 0)
  residual_z = np.sum(np.absolute(residual[i, 2]), axis = 0)

  return residual_u, residual_v, residual_z, total_velocity


# Genetic Algorithm Lama

In [48]:
def print_mutation(P0_temp, Q_temp, results, txt_P0_path, txt_Q_path, txt_result_path, best_fitness, bucket=''):
    P0 = '%14.4g' * len(P0_temp) % tuple(P0_temp)
    Q = '%14.4g' * len(Q_temp) % tuple(Q_temp) 
    res = '%14.4g' * len(results) % results  
    print('\nNew Results (res_u, res_v, res_z, sum_Q, sum_P0): %s\n' % (res))

    best_fitness_path = '/home/icodes/Abel/Optimisasi_Tugas_Akhir/Object_Matching/Tuning_EKF_GA/GA_11082020_best.txt'
    
    with open(txt_P0_path, 'a') as f:  # append result
        f.write(P0 + '\n')
    with open(txt_Q_path, 'a') as g:  # append result
        g.write(Q + '\n')
    with open(txt_result_path, 'a') as h:  # append result
        h.write(res + '\n')
        
    x_P0 = np.unique(np.loadtxt(txt_P0_path, ndmin=2), axis=0)  # load unique rows
    x_Q = np.unique(np.loadtxt(txt_Q_path, ndmin=2), axis=0)
    x_res = np.unique(np.loadtxt(txt_result_path, ndmin=2), axis=0)
    
    np.savetxt(txt_P0_path, x_P0[np.argsort(-cost_func(x_res))], '%10.5g')  # save sort by fitness
    np.savetxt(txt_Q_path, x_Q[np.argsort(-cost_func(x_res))], '%10.5g')
    np.savetxt(txt_result_path, x_res[np.argsort(-cost_func(x_res))], '%10.5g')
    
    with open(best_fitness_path, 'a') as q:  # append result
        q.write(str(best_fitness) + '\n')

In [44]:
def plot_evolution_results(P0_temp, Q_temp, results, txt_P0_path, txt_Q_path, txt_result_path, png_P0_path, png_Q_path):
    P0 = np.loadtxt(txt_P0_path, ndmin=2)
    Q = np.loadtxt(txt_Q_path, ndmin=2)
    res = np.loadtxt(txt_result_path, ndmin=2)
    f = cost_func(res)
    weights = (f - f.min()) ** 2  # for weighted results
    fig_P0 = plt.figure(figsize=(15, 10))
    matplotlib.rc('font', **{'size': 8})
    
    for i, k in enumerate(P0_temp):
        y = P0[:, i]
        mu = y[f.argmax()]  # best single result
        plt.subplot(6, 6, i + 1)
        plt.plot(mu, f.max(), 'o', markersize=10)
        plt.plot(y, f, '.')
        plt.title('%.3g' % (mu), fontdict={'size': 9})  # limit to 40 characters

    fig_P0.tight_layout()
    plt.savefig(png_P0_path, dpi=200)
    plt.close()
    
    fig_Q = plt.figure(figsize=(15, 10))
    for i, k in enumerate(Q_temp):
        y = Q[:, i]
        mu = y[f.argmax()]  # best single result
        plt.subplot(6, 6, i + 1)
        plt.plot(mu, f.max(), 'o', markersize=10)
        plt.plot(y, f, '.')
        plt.title('%.3g' % (mu), fontdict={'size': 9})  # limit to 40 characters

    fig_Q.tight_layout()
    plt.savefig(png_Q_path, dpi=200)
    plt.close()

In [45]:
# Cost Function
def cost_func(x):
  w = [0.2, 0.2, 0.3, 0.3, 0.1, 0.1]
  return 1/x

In [46]:
# Initialization

Q_temp = np.empty((36,))
P0_temp = np.empty((36,))

npr = np.random
keys = ['Q', 'P_0']
limits = [(-1000, 1000), (-1000, 1000)]
for k, v in zip(keys, limits):
    if k == 'Q':
        Q_temp = (v[1]-v[0]) * npr.rand(36,) + v[0]
    elif k =='P_0':
        P0_temp = (v[1]-v[0]) * npr.rand(36,) + v[0]

In [49]:
# EVOLVE
max_gen = 300
txt_P0_path = "/home/icodes/Abel/Optimisasi_Tugas_Akhir/Object_Matching/Tuning_EKF_GA/GA_P0_11082020.txt"
txt_Q_path = "/home/icodes/Abel/Optimisasi_Tugas_Akhir/Object_Matching/Tuning_EKF_GA/GA_Q_11082020.txt"
txt_result_path = "/home/icodes/Abel/Optimisasi_Tugas_Akhir/Object_Matching/Tuning_EKF_GA/GA_Result_11082020.txt"
png_P0_path = "/home/icodes/Abel/Optimisasi_Tugas_Akhir/Object_Matching/Tuning_EKF_GA/GA_P0_11082020.png"
png_Q_path = "/home/icodes/Abel/Optimisasi_Tugas_Akhir/Object_Matching/Tuning_EKF_GA/GA_Q_11082020.png"

best_fitness = 0
for i in range(max_gen):
  print("----------------------------------------------------------------------------------------------------")
  print("GENERATION: "+ str(i+1) + " / " + str(max_gen))
  if os.path.exists(txt_P0_path) and os.path.exists(txt_Q_path):
    parent = 'weighted'
    P0 = np.loadtxt(txt_P0_path, ndmin=2)
    Q = np.loadtxt(txt_Q_path, ndmin=2)
    res = np.loadtxt(txt_result_path, ndmin=2)
    n = min(10, len(P0))
    
    best_fitness = max(cost_func(res))
    print("Best Cost Function")
    print(best_fitness)
    
    P0 = P0[np.argsort(-cost_func(res))][:n]
    Q = Q[np.argsort(-cost_func(res))][:n]
    res = res[np.argsort(-cost_func(res))][:n]
    w = cost_func(res) - cost_func(res).min()
    if len(P0) == 1 and len(Q)==1:
      P0 = P0[random.choices(range(n), weights=w)[0]]
      Q = Q[random.choices(range(n), weights=w)[0]]
    elif parent == 'weighted':
      P0 = (P0 * w.reshape(n,1)).sum(0)/w.sum()
      Q = (Q * w.reshape(n,1)).sum(0)/w.sum()
    
    # Mutate
    mp, s = 0.9, 0.2
    npr = np.random
    npr.seed(int(time.time()))
    g = np.ones((36,))
    ng = len(g)
    v_1 = np.ones(ng)
    v_2 = np.ones(ng)
    while all(v_1==1):
      v_1 = (g * (npr.random(ng) < mp) * npr.randn(ng) * npr.random() * s + 1).clip(0.2, 5.0)
    while all(v_2==1):
      v_2 = (g * (npr.random(ng) < mp) * npr.randn(ng) * npr.random() * s + 1).clip(0.2, 5.0)
    
    for i, k in enumerate(Q_temp):
      Q_temp[i] = Q[i]* v_1[i]
    for i, k in enumerate(P0_temp):
      P0_temp[i] = P0[i]* v_2[i]

  rmse = simulate(yolo, cx, fx, cy, fy, Q_temp.reshape(6,6), data, P0_temp.reshape(6,6)
    
  print_mutation(P0_temp, Q_temp, rmse, txt_P0_path, txt_Q_path, txt_result_path, best_fitness)
  
  plot_evolution_results(P0_temp, Q_temp, rmse, txt_P0_path, txt_Q_path, txt_result_path, png_P0_path, png_Q_path)

----------------------------------------------------------------------------------------------------
GENERATION: 1 / 300

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      2.163e+04     1.033e+05      1.39e+05     2.041e+09      1.95e+04     2.253e+04

----------------------------------------------------------------------------------------------------
GENERATION: 2 / 300
Best Cost Function
2.3597356059955352e-05

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      9.325e+04     1.866e+05     6.167e+05     2.578e+11     1.929e+04     2.313e+04

----------------------------------------------------------------------------------------------------
GENERATION: 3 / 300
Best Cost Function
2.3597356059955352e-05

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      1.042e+07     6.727e+06     3.485e+06     1.441e+12     1.895e+04      2.32e+04

----------------------------------------------------------------------------------------------------
GENERATION: 4 / 300
Best Cost Function
2.

----------------------------------------------------------------------------------------------------
GENERATION: 29 / 300
Best Cost Function
19.174657879110107

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      2.178e+06     9.569e+05     1.762e+05     4.976e+10     1.936e+04      2.33e+04

----------------------------------------------------------------------------------------------------
GENERATION: 30 / 300
Best Cost Function
19.174657879110107

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          6.614         3.241        0.5604         961.4     1.967e+04     2.419e+04

----------------------------------------------------------------------------------------------------
GENERATION: 31 / 300
Best Cost Function
19.174657879110107

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          12.23         4.017        0.4909          1359     1.945e+04     2.392e+04

----------------------------------------------------------------------------------------------------
GENERATI

----------------------------------------------------------------------------------------------------
GENERATION: 57 / 300
Best Cost Function
19.174657879110107

New Results (res_u, res_v, res_z, sum_Q, sum_P0):           8.46          11.5        0.3333          1312     1.923e+04     2.407e+04

----------------------------------------------------------------------------------------------------
GENERATION: 58 / 300
Best Cost Function
19.174657879110107

New Results (res_u, res_v, res_z, sum_Q, sum_P0):           9.62         2.571       0.01621         897.5     1.857e+04     2.225e+04

----------------------------------------------------------------------------------------------------
GENERATION: 59 / 300
Best Cost Function
19.174657879110107

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          7.657        0.8926       0.03937          1874     1.927e+04     2.272e+04

----------------------------------------------------------------------------------------------------
GENERATI

----------------------------------------------------------------------------------------------------
GENERATION: 85 / 300
Best Cost Function
36.264584259641

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          7.219         1.413        0.5717          6147     1.927e+04     2.329e+04

----------------------------------------------------------------------------------------------------
GENERATION: 86 / 300
Best Cost Function
36.264584259641

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          14.57         3.023        0.3415          2629     1.899e+04     2.385e+04

----------------------------------------------------------------------------------------------------
GENERATION: 87 / 300
Best Cost Function
36.264584259641

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      1.305e+07     1.435e+07      1.02e+07      8.11e+11      1.81e+04      2.47e+04

----------------------------------------------------------------------------------------------------
GENERATION: 88 / 

----------------------------------------------------------------------------------------------------
GENERATION: 113 / 300
Best Cost Function
36.264584259641

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          6.327         1.209      0.004087          1590     1.933e+04     2.269e+04

----------------------------------------------------------------------------------------------------
GENERATION: 114 / 300
Best Cost Function
73.60070998678376

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          6.725         1.873         0.595          1143     1.986e+04     2.392e+04

----------------------------------------------------------------------------------------------------
GENERATION: 115 / 300
Best Cost Function
73.60070998678376

New Results (res_u, res_v, res_z, sum_Q, sum_P0):           1857     3.108e+05     9.779e+05     1.065e+13     1.922e+04     2.316e+04

----------------------------------------------------------------------------------------------------
GENERATION

...

----------------------------------------------------------------------------------------------------
GENERATION: 169 / 300
Best Cost Function
73.60070998678376

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          7.677        0.6487        0.1737          1340     1.929e+04     2.369e+04

----------------------------------------------------------------------------------------------------
GENERATION: 170 / 300
Best Cost Function
73.60070998678376

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          4.959         19.29         1.548          1679     1.916e+04     2.428e+04

----------------------------------------------------------------------------------------------------
GENERATION: 171 / 300
Best Cost Function
73.60070998678376

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          9.091         4.662        0.5587          6880     1.987e+04     2.546e+04

----------------------------------------------------------------------------------------------------
GENERATI

----------------------------------------------------------------------------------------------------
GENERATION: 197 / 300
Best Cost Function
73.60070998678376

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      1.931e+04     1.232e+04     1.408e+05      8.74e+09     1.873e+04     2.323e+04

----------------------------------------------------------------------------------------------------
GENERATION: 198 / 300
Best Cost Function
73.60070998678376

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          8.119         1.742       0.02633          1654     1.895e+04     2.281e+04

----------------------------------------------------------------------------------------------------
GENERATION: 199 / 300
Best Cost Function
73.60070998678376

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      2.499e+09     1.821e+08     3.196e+08       2.2e+15     1.829e+04     2.383e+04

----------------------------------------------------------------------------------------------------
GENERATI

----------------------------------------------------------------------------------------------------
GENERATION: 225 / 300
Best Cost Function
118.82111687312504

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      4.533e+05     2.799e+05     7.809e+04     1.648e+10     1.947e+04     2.389e+04

----------------------------------------------------------------------------------------------------
GENERATION: 226 / 300
Best Cost Function
118.82111687312504

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      5.842e+09     1.616e+09     4.552e+08     2.655e+15     1.958e+04     2.454e+04

----------------------------------------------------------------------------------------------------
GENERATION: 227 / 300
Best Cost Function
118.82111687312504

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      6.518e+04         458.1     7.683e+05     5.125e+12     1.877e+04      2.44e+04

----------------------------------------------------------------------------------------------------
GENER

----------------------------------------------------------------------------------------------------
GENERATION: 253 / 300
Best Cost Function
118.82111687312504

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      1.196e+07     1.114e+07     3.319e+06     2.207e+13     1.962e+04     2.423e+04

----------------------------------------------------------------------------------------------------
GENERATION: 254 / 300
Best Cost Function
118.82111687312504

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      1.648e+05     4.174e+04     2.119e+05     7.857e+10       1.9e+04     2.437e+04

----------------------------------------------------------------------------------------------------
GENERATION: 255 / 300
Best Cost Function
118.82111687312504

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      1.114e+06     4.622e+06     1.623e+06     3.774e+11     1.917e+04     2.428e+04

----------------------------------------------------------------------------------------------------
GENER

----------------------------------------------------------------------------------------------------
GENERATION: 281 / 300
Best Cost Function
118.82111687312504

New Results (res_u, res_v, res_z, sum_Q, sum_P0):          11.49         2.403        0.4978         809.8     1.925e+04     2.502e+04

----------------------------------------------------------------------------------------------------
GENERATION: 282 / 300
Best Cost Function
118.82111687312504

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      4.007e+07     1.145e+08     4.805e+07      1.95e+13      1.92e+04     2.407e+04

----------------------------------------------------------------------------------------------------
GENERATION: 283 / 300
Best Cost Function
118.82111687312504

New Results (res_u, res_v, res_z, sum_Q, sum_P0):      4.947e+04     2.551e+04     7.275e+05     1.164e+12     1.906e+04     2.378e+04

----------------------------------------------------------------------------------------------------
GENER

In [51]:
a = np.loadtxt('/home/icodes/Abel/Optimisasi_Tugas_Akhir/Object_Matching/Tuning_EKF_GA/GA_Q_11082020.txt')
a[1].reshape(6,6)

array([[   11.17,  -211.6 ,   638.3 ,   236.7 ,   773.1 ,  -153.7 ],
       [   60.55,  -156.6 ,   721.3 ,    87.24,   -86.42,  -224.4 ],
       [ -324.1 ,  -342.5 ,  -377.9 ,  -308.5 ,  -253.9 ,  1022.  ],
       [ 1397.  ,  -266.2 ,  -194.6 , -1169.  ,  -438.7 ,  -763.  ],
       [  279.1 ,   124.3 ,  -484.1 ,   302.  ,   291.8 ,  2243.  ],
       [ -521.2 ,  -426.3 ,   132.6 ,  2883.  , -1233.  ,   397.5 ]])